In [1]:
%load_ext dotenv
%dotenv

import os

%cd {os.getenv("PROJECT_PATH") or "."}

%load_ext autoreload
%autoreload 1

from IPython.display import display

/data/hdd1/users/akouk/ChemFlow/ChemFlow


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
from absl import logging
from tqdm.notebook import tqdm, trange
from timeit import default_timer as timer
import pickle
from collections import defaultdict

logging.set_verbosity(logging.INFO)

In [3]:
from pandarallel import pandarallel

pandarallel.initialize(
    nb_workers=os.cpu_count(),
    progress_bar=True,
    verbose=0
)

In [4]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)

In [5]:
from rdkit import Chem

from src.utils.scores import *
from src.vae import load_vae
from src.pinn.pde import load_wavepde
from src.pinn import VAEGenerator

Found local copy...
Found local copy...
Found local copy...


In [6]:
# df = pd.read_csv('data/interim/corr/hj_unsup_0.1_binding_affinity.csv', index_col=0)
df = pd.read_csv('data/interim/corr/wave_unsup_0.1.csv', index_col=0)
# df = pd.read_csv('data/interim/corr/vaepde_unsup_0.1.csv', index_col=0)

# df['1err'] = df['1err'].apply(lambda x: min(x, 0))
# df['2iik'] = df['2iik'].apply(lambda x: min(x, 0))

show_df(df)

,k_idx,idx,t,smiles,index,plogp,sa,qed,drd2,jnk3,gsk3b,uplogp
0,0,0,0,CNC=C(COCN)/C#CNCCCOC=CN=CC(C)C[NH1]C(F)(C)SF,0,-3.264532,5.654372,0.061369,0.010631,0.03,0.02,-3.646072
1,0,1,0,O[NH1+1](C#N)CCCNC(=CC=C1C)N(OC=CC=C1Cl)C,1,-17.992777,5.700959,0.401590,0.005917,0.00,0.02,-5.678579
2,0,2,0,C=C(C)CCC=NCNCNNC=CCC=CC=O,2,-2.033098,4.432049,0.095390,0.004846,0.00,0.06,-2.760849
3,0,3,0,C1COC=C(OC)N2N=NS#CSCCC(=O)C=C2C1CNO,3,-20.713743,5.780352,0.590009,0.003486,0.00,0.05,-5.509352
4,0,4,0,CCCCCCC=C(C)NCCNNCCNN(CC=CC)OCOC=CSN=C=C,4,-0.813712,4.657353,0.037397,0.012424,0.07,0.10,-0.849453


(100000, 12)


In [8]:
#print the nmae of the columsn from 4-11
print(df.columns[5:12])

Index(['plogp', 'sa', 'qed', 'drd2', 'jnk3', 'gsk3b', 'uplogp'], dtype='object')


In [10]:
PROPS = ["plogp", "sa", "qed", "drd2", "jnk3", "gsk3b", "uplogp"]

In [11]:
from scipy.stats import pearsonr, spearmanr, linregress

def fun(x:pd.DataFrame):
    results = []
    # for prop in PROPS:
    for j in range(5, 12):
    #for j in range(5, 7):
        r = [
            x.iloc[i, j] for i in range(10)
        ]
        results.append(pearsonr(range(10), r)[0])
    s = pd.Series(results, index=PROPS)
    
    s['invalid'] = x['smiles'].nunique()==1
    
    return s

_df = df.groupby(['k_idx', 'idx']).apply(fun)
# _df = df.groupby(['k_idx', 'idx']).apply(fun)

show_df(_df)

plogp        sa       qed      drd2      jnk3     gsk3b  \
k_idx idx                                                               
0     0    0.864114 -0.533383 -0.224718  0.917161 -0.646362  0.246139   
      1    0.391131  0.814033 -0.299118 -0.898494  0.383167 -0.129165   
      2    0.916382 -0.913737  0.902030 -0.837120  0.797724 -0.878595   
      3   -0.907017  0.720772 -0.911478  0.675984       NaN  0.815163   
      4   -0.853633  0.867615  0.821528 -0.828810 -0.696311 -0.798723   

             uplogp  invalid  
k_idx idx                     
0     0    0.833024    False  
      1   -0.794191    False  
      2    0.908815    False  
      3   -0.875643    False  
      4   -0.839349    False

(10000, 8)


In [13]:
# df_result = _df.query('not invalid').fillna(0)
df_result = _df.fillna(0)

# show_df(df_result)

print(df_result.shape)

(10000, 8)


In [23]:
#df_result.groupby('k_idx').mean()

In [15]:
for prop in PROPS:
    max = df_result[prop].idxmax()[0]
    print(f"{prop}: {max}")

plogp: 2
sa: 6
qed: 0
drd2: 3
jnk3: 5
gsk3b: 5
uplogp: 6


In [16]:
#find the row with highest 1err 
max_1err = df_result['1err'].idxmax()
print(max_1err[0])
max_2iik = df_result['2iik'].idxmax()
print(max_2iik[0])

KeyError: '1err'

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# List of property names
PROPS = ["plogp", "sa", "qed", "drd2", "jnk3", "gsk3b", "uplogp"]

def fun(x: pd.DataFrame):
    results = []
    n = len(x)  # ideally 10 steps per group
    # For each property column (columns 5 through 11 in your DataFrame)
    for j in range(5, 12):
        # Extract the property values from the group.
        r = list(x.iloc[:n, j])
        # Compute Pearson correlation between step indices (0,1,...,n-1) and property values.
        corr, _ = pearsonr(range(n), r)
        results.append(corr)
    # Create a Series with correlation coefficients for each property.
    s = pd.Series(results, index=PROPS)
    # Flag groups where all smiles are identical (could indicate a failed traversal)
    s['invalid'] = (x['smiles'].nunique() == 1)
    return s

# Load the CSV file (adjust the path if needed)
df = pd.read_csv('data/interim/corr/wave_unsup_0.1.csv', index_col=0)

# Group by k_idx and idx, then apply the correlation function
_df = df.groupby(['k_idx', 'idx']).apply(fun)

# Display the results (for example, the first few rows)
print(_df.head())

              plogp        sa       qed      drd2      jnk3     gsk3b  \
k_idx idx                                                               
0     0    0.864114 -0.533383 -0.224718  0.917161 -0.646362  0.246139   
      1    0.391131  0.814033 -0.299118 -0.898494  0.383167 -0.129165   
      2    0.916382 -0.913737  0.902030 -0.837120  0.797724 -0.878595   
      3   -0.907017  0.720772 -0.911478  0.675984       NaN  0.815163   
      4   -0.853633  0.867615  0.821528 -0.828810 -0.696311 -0.798723   

             uplogp  invalid  
k_idx idx                     
0     0    0.833024    False  
      1   -0.794191    False  
      2    0.908815    False  
      3   -0.875643    False  
      4   -0.839349    False  
